In [ ]:
!pip install bayesian-optimization

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn  as sns

from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
df= pd.read_csv("/content/drive/MyDrive/ML/heart.csv")

In [ ]:
df

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [ ]:
df.isnull().sum()

,0
age,0
sex,0
cp,0
trtbps,0
chol,0
fbs,0
restecg,0
thalachh,0
exng,0
oldpeak,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [ ]:
df1 = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df1.head(10)

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,52,1,1,128,205,1,1,184,0,0.0,2,0,2,1
1,42,0,2,120,209,0,1,173,0,0.0,1,0,2,1
2,67,1,2,152,212,0,0,150,0,0.8,1,0,3,0
3,54,0,2,160,201,0,1,163,0,0.0,2,1,2,1
4,49,0,1,134,271,0,1,162,0,0.0,1,0,2,1
5,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0
6,41,1,0,110,172,0,0,158,0,0.0,2,0,3,0
7,57,1,0,110,201,0,1,126,1,1.5,1,0,1,1
8,60,1,2,140,185,0,0,155,0,3.0,1,0,2,0
9,62,0,0,140,394,0,0,157,0,1.2,1,0,2,1


In [ ]:
X= df1.drop(columns=['output'])
y= df1['output']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler1= MinMaxScaler()
MinMax_scaled_X= scaler1.fit_transform(X)
MinMax_scaled_X

array([[0.47916667, 1.        , 0.33333333, ..., 1.        , 0.        ,
        0.66666667],
       [0.27083333, 0.        , 0.66666667, ..., 0.5       , 0.        ,
        0.66666667],
       [0.79166667, 1.        , 0.66666667, ..., 0.5       , 0.        ,
        1.        ],
       ...,
       [0.77083333, 0.        , 0.66666667, ..., 0.5       , 0.25      ,
        0.66666667],
       [0.45833333, 1.        , 0.66666667, ..., 1.        , 0.        ,
        0.66666667],
       [0.5625    , 1.        , 0.33333333, ..., 0.        , 0.        ,
        0.66666667]])

<font color= "lime"> <h2> Random Search

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(MinMax_scaled_X,y,test_size= 20,random_state=0)

In [ ]:
rf_param_grid = {
                 'max_depth' : list(range(8,100,10)),
                 'n_estimators': list(range(50,2000,100)),
                 'min_samples_split': [2,6,8],
                 'min_samples_leaf': [2,6,8],
                 'bootstrap': [True, False]
                 }

In [ ]:
rf_param_grid

{'max_depth': [8, 18, 28, 38, 48, 58, 68, 78, 88, 98],
 'n_estimators': [50,
  150,
  250,
  350,
  450,
  550,
  650,
  750,
  850,
  950,
  1050,
  1150,
  1250,
  1350,
  1450,
  1550,
  1650,
  1750,
  1850,
  1950],
 'min_samples_split': [2, 6, 8],
 'min_samples_leaf': [2, 6, 8],
 'bootstrap': [True, False]}

In [ ]:
m = RandomForestClassifier(n_jobs= 4)   # use -1 for full CPU utilization

In [ ]:
# m_r = GridSearchCV(param_grid=rf_param_grid, estimator = m, scoring = "accuracy", cv = 4)

In [ ]:
m_r = RandomizedSearchCV(param_distributions=rf_param_grid, estimator = m, scoring = "accuracy", cv = 4,n_jobs=-1,n_iter=10)

In [ ]:
%time m_r.fit(X_train,y_train)

CPU times: user 3.88 s, sys: 447 ms, total: 4.32 s
Wall time: 1min 20s


RandomizedSearchCV(cv=4, estimator=RandomForestClassifier(n_jobs=4), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [8, 18, 28, 38, 48, 58, 68,
                                                      78, 88, 98],
                                        'min_samples_leaf': [2, 6, 8],
                                        'min_samples_split': [2, 6, 8],
                                        'n_estimators': [50, 150, 250, 350, 450,
                                                         550, 650, 750, 850,
                                                         950, 1050, 1150, 1250,
                                                         1350, 1450, 1550, 1650,
                                                         1750, 1850, 1950]},
                   scoring='accuracy')

In [ ]:
m_r.best_params_

{'n_estimators': 1650,
 'min_samples_split': 8,
 'min_samples_leaf': 6,
 'max_depth': 8,
 'bootstrap': False}

In [ ]:
for param, score in zip(m_r.cv_results_['params'], m_r.cv_results_['mean_test_score']):
    print(param, score)

{'n_estimators': 150, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_depth': 48, 'bootstrap': True} 0.8056841046277665
{'n_estimators': 1050, 'min_samples_split': 6, 'min_samples_leaf': 6, 'max_depth': 98, 'bootstrap': True} 0.8092555331991952
{'n_estimators': 450, 'min_samples_split': 6, 'min_samples_leaf': 8, 'max_depth': 18, 'bootstrap': False} 0.80920523138833
{'n_estimators': 1650, 'min_samples_split': 8, 'min_samples_leaf': 6, 'max_depth': 8, 'bootstrap': False} 0.8162977867203219
{'n_estimators': 1650, 'min_samples_split': 8, 'min_samples_leaf': 8, 'max_depth': 98, 'bootstrap': True} 0.7986418511066398
{'n_estimators': 1550, 'min_samples_split': 2, 'min_samples_leaf': 8, 'max_depth': 48, 'bootstrap': False} 0.80920523138833
{'n_estimators': 1450, 'min_samples_split': 6, 'min_samples_leaf': 6, 'max_depth': 18, 'bootstrap': True} 0.8057344064386319
{'n_estimators': 1150, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 58, 'bootstrap': True} 0.8056841046277665
{'n_

In [ ]:
rf_bp = m_r.best_params_

In [ ]:
rf_classifier=RandomForestClassifier(n_estimators=rf_bp["n_estimators"],
                                     min_samples_split=rf_bp['min_samples_split'],
                                     min_samples_leaf=rf_bp['min_samples_leaf'],
                                     max_depth=rf_bp['max_depth'],
                                     bootstrap=rf_bp['bootstrap'])

In [ ]:
rf_classifier.fit(X_train,y_train)

RandomForestClassifier(bootstrap=False, max_depth=8, min_samples_leaf=6,
                       min_samples_split=8, n_estimators=1650)

In [ ]:
y_pred_randomCV = rf_classifier.predict(X_test)

In [ ]:
y_pred_randomCV

array([0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1])

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

y_pred_randomCV = rf_classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred_randomCV))
print(accuracy_score(y_test,y_pred_randomCV))
print(classification_report(y_test,y_pred_randomCV))
acc = accuracy_score(y_test,y_pred_randomCV)

[[ 8  1]
 [ 0 11]]
0.95
              precision    recall  f1-score   support

           0       1.00      0.89      0.94         9
           1       0.92      1.00      0.96        11

    accuracy                           0.95        20
   macro avg       0.96      0.94      0.95        20
weighted avg       0.95      0.95      0.95        20



In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score



print("accuracy: ",accuracy_score(y_test,y_pred_randomCV))
print("precision_score: ",precision_score(y_test,y_pred_randomCV))
print("recall_score: ",recall_score(y_test,y_pred_randomCV))
print("f1_score: ",f1_score(y_test,y_pred_randomCV))
print("roc_auc_score: ",roc_auc_score(y_test,y_pred_randomCV))

accuracy:  0.95
precision_score:  0.9166666666666666
recall_score:  1.0
f1_score:  0.9565217391304348
roc_auc_score:  0.9444444444444444


<font color= "lime"> <h2> Baysian Optimization

Bayesian Optimization uses probability to find the minimum of a function. The final airm is to find the input value to a function which can give us the lowest possible output value. There are 3 main paramaters in Bayesian Optimization.
1. Objective function: loss function to minimize
2. Domain Space: range of input values
3. Optimization algorithm: search algorithm
Bayesian Optimization builds a probability model of the objective function and uses it to select hyperparameter to evaluate in the true objective function


description:

hp --> for defining hyperparameter values
STATUS_OK --> evaluation successfully done or not
Trials --> used for minimizing objective function
fmin --> minimize function
TPE (Tree-structured Parzen Estimator) --> Search algorithm

In [ ]:
!pip install hyperopt

In [ ]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

In [ ]:
space = {'criterion': hp.choice('criterion', ['gini', 'entropy']),
        'n_estimators': hp.choice('n_estimators',[ 10,50,100,200]),
        'min_samples_leaf': hp.choice('min_samples_leaf',[ 1,2,3]),
        'min_samples_split' : hp.choice ('min_samples_split',[ 2,3,4]),
        'max_depth' : hp.choice ('max_depth', [3,5,10]),
        'max_features': hp.choice('max_features', ['sqrt', 'log2', None]),
        'max_leaf_nodes' : hp.choice ('max_leaf_nodes',[2,3,4, None])
    }

In [ ]:
space

{'criterion': <hyperopt.pyll.base.Apply at 0x7c8d22b1b3a0>,
 'n_estimators': <hyperopt.pyll.base.Apply at 0x7c8d22b1a4d0>,
 'min_samples_leaf': <hyperopt.pyll.base.Apply at 0x7c8d22b19c30>,
 'min_samples_split': <hyperopt.pyll.base.Apply at 0x7c8d22b1b490>,
 'max_depth': <hyperopt.pyll.base.Apply at 0x7c8d22b1a290>,
 'max_features': <hyperopt.pyll.base.Apply at 0x7c8d22b1a110>,
 'max_leaf_nodes': <hyperopt.pyll.base.Apply at 0x7c8d22b19f60>}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


def objective(space):

  RF_classifier = RandomForestClassifier(criterion= space['criterion'],
                                           n_estimators= space['n_estimators'],
                                           min_samples_split= space['min_samples_split'],
                                           min_samples_leaf= space['min_samples_leaf'],
                                           max_depth= space['max_depth'],
                                           max_features= space['max_features'],
                                           max_leaf_nodes= space['max_leaf_nodes'])

  accuracy = cross_val_score(RF_classifier, X_train, y_train, cv = 5).mean()

  # We aim to maximize accuracy, therefore we return it as a negative value
  return {'loss': -accuracy, 'status': STATUS_OK }

In [ ]:
trials = Trials()
best = fmin(fn= objective,      # fmin minimize the objective function fn
            space= space,
            algo= tpe.suggest,    # tpe.suggest refers to the process of suggesting the next set of hyperparameters using tpe algo
            max_evals = 80,
            trials= trials)
best

100%|██████████| 80/80 [01:23<00:00,  1.04s/trial, best loss: -0.8375313283208021]


{'criterion': 0,
 'max_depth': 0,
 'max_features': 1,
 'max_leaf_nodes': 3,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1}

In [ ]:
# optcome is index sometimes. So we need to find the index value by using mapping (inverse key value)

cri = {0:'gini', 1:'entropy'}
est= {0:10, 1:50, 2:100, 3:200}
min_split = {0:2, 1:3, 2:4}
min_leaf= {0:1, 1:2, 2:3}
depth= {0:3, 1:5, 2:10}
fea = {0:'sqrt', 1:'log2', 2:None}
max_leaf= {0:2, 1:3, 2:4, 3: None}


In [ ]:
trainedRF = RandomForestClassifier(   criterion= cri[best['criterion']],                          # best['criterion'] = 1     cri[1]= "entropy"
                                      n_estimators= est[best['n_estimators']],                    # best['n_estimators']= 3   est[3]= 200
                                      min_samples_split= min_split[best['min_samples_split']],
                                      min_samples_leaf= best['min_samples_leaf'],
                                      max_depth= depth[best['max_depth']],                # best['max_depth'] =1      depth[1] =5
                                      max_features= fea[best['max_features']],
                                      max_leaf_nodes= best['max_leaf_nodes']).fit(X_train,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

predictionGB = trainedRF.predict(X_test)
print(confusion_matrix(y_test,predictionGB))
print(accuracy_score(y_test,predictionGB))
print(classification_report(y_test,predictionGB))
acc = accuracy_score(y_test,predictionGB)

[[ 8  1]
 [ 0 11]]
0.95
              precision    recall  f1-score   support

           0       1.00      0.89      0.94         9
           1       0.92      1.00      0.96        11

    accuracy                           0.95        20
   macro avg       0.96      0.94      0.95        20
weighted avg       0.95      0.95      0.95        20



In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score



print("accuracy: ",accuracy_score(y_test,predictionGB))
print("precision_score: ",precision_score(y_test,predictionGB))
print("recall_score: ",recall_score(y_test,predictionGB))
print("f1_score: ",f1_score(y_test,predictionGB))
print("roc_auc_score: ",roc_auc_score(y_test,predictionGB))

accuracy:  0.95
precision_score:  0.9166666666666666
recall_score:  1.0
f1_score:  0.9565217391304348
roc_auc_score:  0.9444444444444444
